In [25]:
import numpy as np
#from models import Model, Trainer
from Enviroment import Env
from MCTS import MCTS

class dotdict(dict):
    def __getattr__(self, name):
        return self[name]

env = Env(1,1)


In [7]:
import torch
from torch import Tensor, nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cnv1 = nn.Conv1d(1,6,8)
        self.fc1 = nn.Linear(18, 64)
        self.fc2 = nn.Linear(64, 64)
        self.action_prob_out = nn.Linear(64, 8)
        #self.val0 = nn.Linear(40, 80)
        self.val = nn.Linear(64, 1)

    def forward(self, x):
        #print(x.shape)
        #x = x.view(-1,17)
        #x = self.cnv1(x.view(1, 1,-1))
        # print(x.shape)
        #x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        act_prob = F.softmax(self.action_prob_out(x), dim=-1)
        #val = F.tanh(self.val(x))
        val = self.val(x)
        #val_sum = self.val_sum_out(val)

        return act_prob, val

    def predict(self, x):
        self.eval()
        #x = Variable(Tensor(x))
        act_prob, val = self.forward(x)
        return act_prob.data.numpy(), val.data.numpy()
    
    def get_observation(self, formula, env, time=0):
        reward, state = env.get_observation(formula)
        values_norm = np.array(list(state.values()))
        inp, out, err = env.inp, env.out, reward
        #if np.sum(values_norm) != 0:
        #    values_norm = values_norm / np.sum(values_norm)
        s = inp + out
        inp = inp / s
        out = out / s
        if values_norm.shape[0] != 6:
            print('ERROR', self.result, values_norm.shape, self.one_hot_last(formula, env.action_space).shape)
        time = np.sin(time/100)
        head = np.hstack([np.array([inp, out, err, time]), 
                          values_norm, 
                          self.one_hot_last(formula, env.action_space)
                         ])
        return Variable(Tensor(head))

    def one_hot_last(self, formula, act_sp, n_last=1):
        matrix = np.zeros([n_last * len(act_sp)])
        for i, f in enumerate(formula[-n_last:]):
            k = n_last - min(len(formula), n_last)  # start from last line
            matrix[act_sp.index(f) + (k + i) * 8] = 1
        return matrix

    


def get_batchX(nodes_buc, batch_size=20):
    """remove all nodes that have not history"""
    n = len(nodes_buc)
    index = [
        np.random.randint(0, n - 1)
        for _ in range(batch_size)
    ]
    #print(len(self.replay), index, [self.replay[i] for i in index]   )
    return index#[nodes_buc[i] for i in index]

def clean_unpredict_node(nodes_buc):
    """ 
     each leaf when we visit has 8 childrens, 
     but most of it never choose and we do not reward on it. 
     We should remowe all those leaf. ## what about terminal??
    """
    new_buck = []
    for node in nodes_buc:
        if len(node.history_data['next_node_ind']):
            new_buck.append(node)
    return new_buck

def train_model(Xa,yr,yp, model, batch_size=30, net_iters=200):
    optimizer = optim.SGD(model.parameters(), lr=0.1)#, momentum=0.03, weight_decay=0.1)
    loss_backet = []
    #validate_back = []
    for i in range(net_iters):
        #batch = #self.get_batch(nodes_buc, batch_size=batch_size or self.batch_size)
        # print(batch)
        ind = get_batch(Xa)

        X, real_reward, real_prob = Xa[ind], yr[ind], yp[ind]#self.transform_bach_as_input(batch, model)


        #print(i, real_prob.shape)
        model.train()
        #for x, rr, rp in zip(X, real_reward, real_prob):
            #print(xx, rrr,rpp)
        optimizer.zero_grad()
        #print(X)
        p_pred, v_pred = model(X)
        # print('pr  ', probability, 'pp  ', p_pred)
        val_loss = torch.mean((Variable(Tensor(real_reward)) - v_pred) ** 2)  # , Variable(Tensor([10]))
        #loss = val_loss - torch.mean(Variable(Tensor(real_prob)) * torch.log(p_pred))
        loss = - torch.mean(Variable(Tensor(real_prob)) * torch.log(p_pred))
        #print(loss, rpp, p_pred)
        loss.backward()
        optimizer.step()
        loss_backet.append(loss.data.numpy())
        
    return loss_backet


In [30]:
model = Model()
args = dotdict({'cpuct':0.5, 'iters':10000})
mcts = MCTS(env, model, args)
val = list(mcts.sampling())
for n in val:
    if n.formula and n.immediate_reward == 1:
        print(n.formula, n.immediate_reward)

iso 1
ieo 1
is1so 1
isBso 1
isAeo 1
iiso 1
isAso 1
isBeo 1
isCeo 1
isCso 1
iieo 1
iAiso 1
is1eo 1
ieAeCsBs1iiAso 1
Biso 1
1iso 1
eiso 1


In [31]:
from sklearn.model_selection import train_test_split

val2 = clean_unpredict_node(val) # from deep down
print(len(val), len(val2))
train_cur, test_cur = val2, val2#train_test_split(val, shuffle=True)

7407 5941


In [42]:
def fill_fin_prob(nodes):
    for i in nodes:
        act_count = np.zeros(8)
        i.fin_prob = act_count + 1/8
        #print(i.formula, len(i.history_data['next_node_ind']))
        for act in i.history_data['next_node_ind']:
            act_count[act] += 1
        if len(i.history_data['next_node_ind']):
            i.fin_prob = act_count / len(i.history_data['next_node_ind'])

In [37]:
for n in val2:
    if n.formula and n.formula[-2:] == 'is':
        print(n.formula, n.immediate_reward, [float('%.2f' % elem) for elem in n.fin_prob])

is 0 [0.03, 0.03, 0.83, 0.04, 0.03, 0.03, 0.01, 0.01]
Ais 0 [0.0, 0.0, 0.0, 0.67, 0.33, 0.0, 0.0, 0.0]
Bis 0 [0.0, 0.0, 0.14, 0.57, 0.29, 0.0, 0.0, 0.0]
Cis 0 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
iis 0 [0.12, 0.03, 0.12, 0.07, 0.24, 0.32, 0.04, 0.05]
1is 0 [0.0, 0.0, 0.14, 0.57, 0.29, 0.0, 0.0, 0.0]
eis 0 [0.0, 0.0, 0.14, 0.57, 0.29, 0.0, 0.0, 0.0]
sis 0 [0.0, 0.0, 0.0, 0.67, 0.33, 0.0, 0.0, 0.0]
isis 0 [0.11, 0.24, 0.02, 0.29, 0.07, 0.11, 0.11, 0.04]
ieAis 0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
isACA1iBCAAsis 0 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
isACA1iBCAAsisis 0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
ieis 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
iAis 0 [0.0, 0.0, 0.2, 0.3, 0.5, 0.0, 0.0, 0.0]
is1sis 0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
i1is 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
isAeis 0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
isAAis 0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
iBis 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
isBis 0 [0.0, 0.0, 0.0, 0.0, 0.

In [34]:
Env.action_space

('1', 'i', 'o', 'A', 'B', 'C', 's', 'e')

# Adwersarial game

In [63]:
model1 =Model()# LSTMTagger()#Model()
model2 =Model()
env1 = Env(1,1)
env2 = Env(1,1)

args = dotdict({'cpuct':0.5, 'iters':1000})
mcts1 = MCTS(env1, model, args)
mcts2 = MCTS(env2, model, args)


def play_game(mcts1, mcts2):
    game_end = 0
    score = [0, 0]
    while not game_end:
        node1 = mcts1.select_best_leaf()
        r1 = node1.immediate_reward

        node2 = mcts2.select_best_leaf()
        r2 = node2.immediate_reward
        #print(len(mcts1.Nodes) == len(mcts2.Nodes))
        

        
        if r1 < 0 or r2 < 0:
            mcts1.propagate(node1)
            mcts2.propagate(node2)
        elif r1 > 0 and r2 > 0:
            mcts1.propagate(node1)
            mcts2.propagate(node2)
        elif r1 > 0:
            node2.immediate_reward = -1
            mcts1.propagate(node1)
            mcts2.propagate(node2)
        elif r2 > 0:
            node1.immediate_reward = -1
            mcts1.propagate(node1)
            mcts2.propagate(node2)
        if r1 != 0 or r2 != 0:
            score[0] += node1.immediate_reward
            score[1] += node2.immediate_reward
            game_end = 1
            mcts1.Nodes[node1.formula] = node1
            mcts2.Nodes[node2.formula] = node2
            break
        
        #if r1>0 or r2>0:
        #    print(node1.formula, game_end)
        #    print(node2.formula, game_end)

        mcts1.Nodes[node1.formula] = node1
        mcts2.Nodes[node2.formula] = node2

        pred_P, pred_R = mcts1.model.predict(mcts1.model.get_observation(node1.formula, mcts1.env, mcts1.iter_timer)) # if simple model self.env.observation
        mcts1.expand(node1, pred_P)

        pred_P, pred_R = mcts2.model.predict(mcts2.model.get_observation(node2.formula, mcts2.env, mcts2.iter_timer)) # if simple model self.env.observation
        mcts2.expand(node2, pred_P)
    
    return score, r1, r2

In [72]:
all_nodes = []
v = [0,0]
for i in range(1):
    mcts1 = MCTS(env1, model1, args)
    mcts2 = MCTS(env2, model2, args)
    score = [0, 0]
    for j in range(2000):
        if not j % 100:
            print(j, end=' ')

        res = play_game(mcts1, mcts2)[0]
        #print('s')
        score[0] += res[0]
        score[1] += res[1]
    #if score[0] > score[1]:
    all_nodes += list(mcts1.Nodes.values())
    #else:
    all_nodes += list(mcts2.Nodes.values())
    v[0] += (score[0]-score[1])
    v[1] += score[0]

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [73]:
v, len(all_nodes), fill_fin_prob(all_nodes)

([1392, -406], 2263, None)

In [77]:
for n in all_nodes:
    if len(n.formula) > 1 and n.formula[-2:] == 'ie':
        print(n.formula, n.immediate_reward, [float('%.2f' % elem) for elem in n.fin_prob])

CAie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
sBCAie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
s11ie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
eiCAie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
CiCAie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ie 0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
BiBAie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
111Aie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
C1iAie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Be1Aie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1isiBBie 0 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
1ie 0 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
iie 0 [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12]
B1CsAsie 0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
Ciie 0 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
eie 0 [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12]
Bie 0 [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12]
sie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ie 0 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
iiie 0 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1C